# "Part I. Root-finding. Newton's iteration."

Write a function which performs Newton's iteration for a given function $f(x)$ with known derivative $f'(x)$. Your function should find the root of $f(x)$ with a predefined absolute accuracy $\epsilon$. 

In [1]:
def newton_iteration(f, fder, x0, eps=1e-5, maxiter=1000):
    """Find a root of $f(x) = 0$ via Newton's iteration starting from x0.
    
    Parameters
    ----------
    f : callable
        The function to find a root of.
    fder : callable
        The derivative of `f`.
    x0 : float
        Initial value for the Newton's iteration.
    eps : float
        The target accuracy. 
        The iteration stops when the distance between successive iterates is below `eps`.
        Default is 1e-5.
    maxiter : int
        The maximum number of iterations (default is 1000.)
        Iterations terminate if the number of iterations exceeds `maxiter`.
        This parameter is only needed to avoid infinite loops if iterations wander off.
        
    Returns
    -------
    x : float
        The estimate for the root.
    niter : int
        The number of iterations.
    """
    x = x0
    for i in range (0,maxiter):
        xn = x - f(x)/fder(x)
        if (abs(xn-x) < eps):
            return xn, i+1
        x = xn
    return x, maxiter

### Test I.1 

Test your implementation on a simple example, $f(x) = x^2 - 1$ or similar. (20% of the total grade)

In [2]:
newton_iteration(lambda x: pow(x,2)-1, lambda x: 2*x, -.7)

(-1.0000000000017693, 4)

### Test I.2

Now consider a function which has a multiple root. Take $f(x) = (x^2 - 1)^2$ as an example. Implement a modified Newton's iteraion,

$$
x_{n+1} = x_{n} - m \frac{f(x_n)}{f'(x_n)}
$$

and vary $m= 1, 2, 3, 4, 5$. Check the number of iterations required for convergence within a fixed $\epsilon$. Are your observations consistent with the expectation that the convergence is quadratic is $m$ equals the multiplicity of the root, and is linear otherwise? (40% of the total grade)

In [3]:
def modifiedNewton(f, fder, x0, m, eps=1e-5, maxiter=1000):
# Same as newton_iteration above, but with included parameter m
    x = x0
    for i in range(0,maxiter):
        xn = x - m*(f(x)/fder(x))
        if (abs(xn-x) < eps):
            return xn, i+1
        xp = x
        x = xn
    return x, maxiter

def f(x):
    return pow(pow(x,2)-1, 2)
def fder(x):
    return 4*pow(x,3)-4*x

for m in range(1,6):
        print(modifiedNewton(f, fder, 0.7, m))

(0.999993656394025, 15)
(1.0000000000017693, 4)
(1.0000028723525891, 17)
(0.6999999999999996, 1000)
(1.6595581105793895, 1000)


The quadratic convergence for $m = 2$ is pretty evident above, compared to the linear convergence for $m = 1,3$. The method does not converge for $m=4,5$.

Below we plot the convergence for different values of $m$ and across different numbers of steps to qualitatively confirm the quadratic vs. linear convergence. The plot may not work depending on your installation, but you can see a nice quadratic shape in the $log$ of the residual for $m=2$, and a linear one otherwise.

In [4]:
import altair as alt
import pandas as pd
import math
data = []

# Calculate the log  of the residual |r - xn| against the number of iterations, for varying m
for m in range(1,4):
    for steps in range(0,20):
        result = modifiedNewton(f, fder, 0.5, m, 1e-5, steps)
        error = abs(1.0 - result[0])
        data.append([steps, math.log(error), m])

# Plot this data
df = pd.DataFrame(data, columns=['Number of Iterations', 'log Distance from Root', 'm='])
df.reset_index()

alt.Chart(df).mark_circle().encode(
    x='Number of Iterations',
    y='log Distance from Root',
    color='m=:N',
)

alt.Chart(...)

# Part II. Fixed-point iteration

Consider the following equation:

$$
\sqrt{x} = \cos{x}
$$

Plot the left-hand side and right-hand side of this equation, and localize the root graphically. Estimate the location of the root by visual inspection of the plot.

Write a function which finds the solution using fixed-point iteration up to a predefined accuracy $\epsilon$. Compare the result to an estimate from a visual inspection.

Next, rewrite the fixed-point problem in the form

$$
x = x - \alpha f(x)
$$

where $\alpha$ is the free parameter. Check the dependence of the number of iterations required for a given $\epsilon$ on $\alpha$. Compare your results to an expectation that the optimal value of $\alpha$ is given by 

$$
\alpha = \frac{2}{m + M}
$$

where $0 < m < |f'(x)| < M$ over the localization interval. (40% of the total grade)

In [5]:
import numpy as np

# Calculate y = sqrt(x)-cos(x) and its derivative for a range of values
x = np.arange(0.0,2.0,0.01)
y = pow(x,0.5)-np.cos(x)
yPrime = 0.5*pow(x,-0.5)+np.sin(x)

# Format this data in so Altair can parse it
data2 = []
for i in range(0,len(x)):
    data2.append([x[i], y[i], 'y = sqrt(x) - cos(x)'])
    data2.append([x[i], yPrime[i], 'y\''])
df2 = pd.DataFrame(data2, columns=['x', 'y', 'Function'])
df2.reset_index()

# Plot this data
alt.Chart(df2).mark_line().encode(
    x=alt.X('x:Q', axis=alt.Axis(offset=-46.15)),
    y='y:Q',
    color='Function:N'
)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in power
  


alt.Chart(...)

Visually, we can estimate the root to be at approximately $x=0.635$. First, a sanity check:

In [6]:
def g(x):
    return pow(x,0.5) - np.cos(x)
def gder(x):
    return 0.5*pow(x,-0.5) + np.sin(x)
g(0.635)

-0.008202823948001425

Pretty good, but let's try to do better with fixed point iteration:

In [7]:
def generalFPI(g, x0, alpha, eps=1e-10, maxiter=1000):
    x = x0
    for i in range(0, maxiter):
        xn = x - alpha*g(x)
        if (abs(xn-x) < eps):
            return xn, i
        x = xn
    return x, maxiter

result = generalFPI(g, 0.635, 1)
root = result[0]
print(result)

(0.6417143708679357, 13)


In [8]:
g(root)

-6.04882810506524e-12

This estimate for the root is about seven orders of magnitude better than the visual estimate &mdash; a pretty good improvement.  
Now we plot the number of steps until convergence ($|x_{n-1} - x_{n}| < \epsilon = 1*10^{-10}$) against the parameter $\alpha$:

In [9]:
# Calculate the number of steps to convergence for a range of alpha parameters
data3 = []
alpha = np.arange(0.1,1.6,0.02)
for a in alpha:
    result = generalFPI(g, 0.635, a)
    data3.append([a,result[1]])

# Plot this data
df3 = pd.DataFrame(data3, columns=['alpha', 'Number of Iterations'])
df3.reset_index()
alt.Chart(df3).mark_line().encode(
    x='alpha',
    y='Number of Iterations'
)

alt.Chart(...)

This gives us an optimum value for $\alpha$ of around $0.82$.  
Since the value of $\frac{dy}{dx} \sqrt{x} - \cos(x)$ is close to $1.2$ when $x$ is close to the root, this agrees with the analysis that $\alpha_{opt} = \frac{2}{m+M}$:

In [10]:
print(gder(root))
print(2/(2*gder(root)))

1.2227342355430186
0.8178392089887776


# Part III. Newton's fractal.

(Not graded). 

Consider the equation

$$
x^3 = 1
$$

It has three solutions in the complex plane, $x_k = \exp(i\, 2\pi k/ 3)$, $k = 0, 1, 2$.

The Newton's iterations converge to one of these solutions, depending on the starting point in the complex plane (to converge to a complex-valued solution, the iteration needs a complex-valued starting point).

Plot the \emph{basins of attraction} of these roots on the complex plane of $x$ (i.e., on the plane $\mathrm{Re}x$ -- $\mathrm{Im}x$). To this end, make a series of calculations, varying the initial conditions on a grid of points. 
Color the grid in three colors, according to the root, to which iterations converged.

In [12]:
import cmath

# Define the equation and its derivative
def h(x):
    return pow(x,3.0) - 1.0
def hder(x):
    return 3.0*pow(x,2.0)

# Returns the a root if the input is very close to that root
roots = [0.0,1.0,2.0]
roots = list(map(lambda k: cmath.exp(1j*2.0*np.pi*k/3.0), roots))
rootNames = {0 : "1 + 0i",
             1 : "-0.5 + 0.866i",
             2 : "-0.5 - 0.866i"}
def isRoot(z):
    for i in range(0,3):
        if (abs(roots[i]-z) < 1e-7):
            return rootNames[i]
    return "No convergence"


# Initialize a grid of values
x = np.arange(-1,1,0.01)
y = np.arange(-1,1,0.01)
rPart, iPart = np.meshgrid(x, y)

# Returns the basin of attraction for given real and imaginary part
def getBasin(r,i):
    z = complex(r,i)
    return isRoot(newton_iteration(h, hder, z, 1e-8)[0])
# Vectorize this function and apply it to our grid
vectorGetBasin = np.vectorize(getBasin)
basins = vectorGetBasin(rPart, iPart)

# Convert this grid to columnar data expected by Altair
df4 = pd.DataFrame({'x': rPart.ravel(),
                     'i': iPart.ravel(),
                     'Root': basins.ravel()})

# Plot this data (warning, may be slow)
alt.data_transformers.disable_max_rows()
w = 600
h = w
s = pow(w/len(rPart),2)

alt.Chart(df4).mark_square(size=s).encode(
    x='x:Q',
    y='i:Q',
    color='Root:N'
).properties(
    width=w,
    height=h
)

alt.Chart(...)